In [4]:
import socket
import threading
import sys
import time
import pickle #Pickle is a module used in python to serialize and deserialize the objects. 
#In socket programming it is used for communicating between the programs both locally and remotely

local = '192.168.159.1', 50001
# The above line incdicates the client ipaddress and port number
con = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

con.bind(local) # Binding the client 
dest = '192.168.159.1', 50000


class tcp_header:
    sequence_number = None
    acknowledgement_number = None
    syn = False
    ack = False
    fin = False
    rst = False
    file = None

    def __init__(self, syn, sequence_number, file, fin):
        self.syn = syn
        self.sequence_number = sequence_number
        self.file = file
        self.fin = fin


class table():
    buffer_list = []
    tcpHeader = tcp_header


count = 100


def End_connection():
    print("sending confirmation ...")
    connectionobject = tcp_header(False, None, None, True)
    data_byte = pickle.dumps(connectionobject) # dumps method in pickle module serialize the data stream
    con.sendto(data_byte, server) # sendto method sends the UDP packet the client socket


def Wait_confirmation():
    global count
    print("Waiting for confirmation ...")
    data, server = con.recvfrom(2048)
    table.tcpHeader = pickle.loads(data) # loads method of pickle module de-serialize the data stream
    while table.tcpHeader.acknowledgement_number != count - 99:
        print("Broadcasting given ...")
        time.sleep(2)
        connectionobject = tcp_header(False, count - 100, table.buffer_list[-1], False)
        data_byte = pickle.dumps(connectionobject) # dumps method in pickle module serialize the data stream
        con.sendto(data_byte, server)
        print("Waiting for confirmation ...")
        data, server = con.recvfrom(2048)
        table.tcpHeader = tcpHeader
        table.tcpHeader = pickle.loads(data)


def Send_file(buffer):
    global count
    connectionobject = tcp_header(False, count, buffer, False)
    table.buffer_list.append(connectionobject)
    data_byte = pickle.dumps(connectionobject) # dumps method in pickle module serialize the data stream
    con.sendto(data_byte, server)
    count = count + 100
    print("Sending file ...")
    buffer = None
    time.sleep(0.3)
    Wait_confirmation()
    return buffer


def Split_file():
    buffer_size = 64000
    f = open("submit.txt", "rb")
    buffer = f.read(buffer_size)
    while buffer:
        Send_file(buffer)
        buffer = f.read(buffer_size)
    End_connection()


try:
    con.settimeout(10.0)
    connectionobject = tcp_header(True, None, None, False)
    data_string = pickle.dumps(connectionobject) # dumps method in pickle module serialize the data stream
    print("Starting connection")
    con.sendto(data_string, dest)
    print("Waiting for connection confirmation")
    data, server = con.recvfrom(2048)
    tcpHeader = pickle.loads(data) # loads method in pickle module deserialize the data stream
    if tcpHeader.syn and tcpHeader.ack:
        Split_file()
except socket.timeout:
    print("TIMEOUT, ending connection ...")

Starting connection
Waiting for connection confirmation
Sending file ...
Waiting for confirmation ...
sending confirmation ...
